<a href="https://colab.research.google.com/github/Gowri-Rk/Machine-Learning/blob/master/Colabs/Alternus%20Vera%20Project/ML_Final_AlternusVera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Final Alternus Vera notebook**

# Problem Statement and Data Narrative

Liar Liar Pants on Fire Dataset Description
It has 3 files test, training and valid.

Each file has 14 columns

Column 1: the ID of the statement ([ID].json).

Column 2: the label.

Column 3: the statement.

Column 4: the subject(s).

Column 5: the speaker.

Column 6: the speaker's job title.

Column 7: the state info.

Column 8: the party affiliation.

Column 9-13: the total credit history count, including the current statement.

Column 14: the context (venue / location of the speech or statement).

Process
Load the Data
Distillation Process
Data Cleaning and Text Preprocessing
Visualization
Chosen feature: Verifiable Authenticity

Merge all features and individual contributions
Form Polynomial Equation

Enrichment Dataset Details
SenticNet5 sensational words corpus
Google News 3million words corpus for spell check
PoliticalFact Fake news and Real News Content
Spam Dictionary

Libraries Used
NLTK
Gensim
Numpy
Pandas
CSV
WordCloud
Seaborn
Scipy
Regualr Expression
Matplotlib
Sklearn
What did I try and What worked?

What did not work?

What alternatives did you try?


# Verifiable Authenticity

In today's world the fake news spreads faster than real news and its important to detect fake news in order to mitigate its negative effects. This project contains analysis and detection of fake news based on verifiable authenticity of the news statement.Authenticity is about how genuine the statement is. 

DataSets used:

1. Liar Liar Dataset: Contains news statements classified as a range of values from True to PantsOnFire. 
2. Kaggle Fake News Dataset: Contains a binary classification of text and author

**Feature: Verifiable Auntenticity**
To classify a news as fake or real, one of the main steps is to find out whether the data can be verified as authentic or not. This involves:
1. Checking the **source** of the news: News obtained from social media or memes tend to be fake. However, news reported by a genuine news publication can be trusted.

**How**: Buzzfeed's dataset on website authenticity (amalgamate)

2. **Credibility**: Some people might give deliberate misinformation to mislead readers. Its important to check the credibilty of the speaker as well as the author of the post.

**How**: Check the past statements made by the speaker to see no. of his false/ true statements

3. **False representation**: These are employed to catch the attention of viewers with false information.

**How**: Cosine similarity with fake_news dataset on the text to match fake news statements

4. **Verify whether other news agencies are reporting the same story**

5. **Verify if the speaker or source stated the same as found in the news**


**Method:** 
Encode the "source" column of liar liar dataset to fall into categories for Fake and Real news based on the source credibility.

**Classification algorithms**: Naive Bayes, Decision Trees, SVM, Linear Regression, Logistic Regression

What worked: Data cleaning, pre-processing and LDA topic modelling, distillation, classification

What didn't work well: Data amalgamation with an other dataset that could enrich the current dataset to provide a stronger feature of authenticity. 

# Imports

In [ ]:
import pandas as pd
import numpy as np
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, roc_auc_score, auc, accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:

# Read the train, test and validation dataset from Liar Dataset 
liar_test_filename =  '/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/liar_dataset/test.tsv'
liar_train_filename = '/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/liar_dataset/train.tsv'
liar_valid_filename = '/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/liar_dataset/valid.tsv'


colnames = ['jsonid', 'label', 'headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context']

df_liar_train = pd.read_csv(liar_train_filename, sep='\t', names = colnames, error_bad_lines=False)
df_liar_test = pd.read_csv(liar_test_filename, sep='\t', names = colnames, error_bad_lines=False)
df_liar_val = pd.read_csv(liar_valid_filename, sep='\t', names = colnames, error_bad_lines=False)

In [ ]:
# Display check the dimensions and the first 2 rows of the file.

print('train dim:',df_liar_train.shape, 'test dim:', df_liar_test.shape)
df_liar_train.iloc[0:2]

train dim: (10240, 14) test dim: (1267, 14)


,jsonid,label,headline_text,subject,speaker,speakerjobtitle,stateinfo,partyaffiliation,barelytruecounts,falsecounts,halftruecounts,mostlytrueocunts,pantsonfirecounts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.


# Data preparation


**Steps included in the preprocessing:**



*   Remove Special Characters and Punctuations

*   Lower case the news
*   Tokenization
*   Remove Stop Words
*   Lemmatization
* Stemming
* Spell Check

## Preprocessing

In [ ]:
import re

def cleaning(raw_news):
    import nltk
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)

In [ ]:
import time
# clean training and test data 
# create new column "tokenized"
t1 = time.time()

# Add the processed data to the original data. 
# Perhaps using apply function would be more elegant and concise than using for loop
df_liar_train['clean'] = df_liar_train["headline_text"].apply(cleaning) 

t2 = time.time()
print("\nTime to clean, tokenize and stem train data: \n", len(df_liar_train), "news:", (t2-t1)/60, "min")

t1 = time.time()
df_liar_test['clean'] = df_liar_test["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem test data: \n", len(df_liar_test), "news:", (t2-t1)/60, "min")

t1 = time.time()
df_liar_val['clean'] = df_liar_val["headline_text"].apply(cleaning)

t2 = time.time()
print("\n\nTime to clean, tokenize and stem valid data: \n", len(df_liar_val), "news:", (t2-t1)/60, "min")


Time to clean, tokenize and stem train data: 
 10240 news: 0.1032333254814148 min


Time to clean, tokenize and stem test data: 
 1267 news: 0.010376377900441488 min


Time to clean, tokenize and stem valid data: 
 1284 news: 0.010247898101806641 min


## Google News corpus word2vec Spell Check


In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('input_data/GoogleNews-vectors-negative300.bin.gz', binary=True)
words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def spell_checker(text):
    all_words = re.findall(r'\w+', text.lower()) # split sentence to words
    spell_checked_text  = []
    for i in range(len(all_words)):
        spell_checked_text.append(correction(all_words[i]))
    return ' '.join(spell_checked_text)

In [ ]:
print("Before: \n", train_news['clean'][0] )
t1 = time.time()
train_news['clean'] = train_news['clean'].apply(spell_checker)
t2 = time.time()
print("\nTime to spell check the train data: \n", len(train_news), "news:", (t2-t1)/60, "min")

print("\nAfter: \n",train_news['clean'][0] )
train_news.head(5)

In [ ]:
t1 = time.time()
test_news['clean'] = test_news['clean'].apply(spell_checker)
test_news.head(5)
t2 = time.time()
print("\nTime to spell check the test data: \n", len(test_news), "news:", (t2-t1)/60, "min")

In [ ]:
t1 = time.time()
valid_news['clean'] = valid_news['clean'].apply(spell_checker)
valid_news.head(5)
t2 = time.time()
print("\nTime to spell check the valid data: \n", len(valid_news), "news:", (t2-t1)/60, "min")

##### Saved the trained dataset into a seperate CSV file 

In [ ]:
train_news.to_csv("input_data/train_processed.csv", sep=',')
test_news.to_csv("input_data/test_processed.csv", sep=',')
valid_news.to_csv("input_data/valid_processed.csv", sep=',')

## Encoding string features to float to be able to run various algorithms

In [ ]:
df_liar_train = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/train_processed.csv")
df_liar_test = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/test_processed.csv")
df_liar_val = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/valid_processed.csv")

Merging training and validation datasets

In [ ]:
# concatenate validation data with training data
# df_liar_train = pd.concat([df_liar_val])


In [ ]:
df_liar_train.head()

We have a lot of columns with string datatypes, we will have to encode that into integer

In [ ]:
#list of categorical features
categoricalVariables = ['label'	,'headline_text',	'subject',	'speaker', 	'speakerjobtitle', 	'stateinfo',	'partyaffiliation', 'context']
numericalVariables = ['barelytruecounts',	'falsecounts',	'halftruecounts',	'mostlytrueocunts',	'pantsonfirecounts']

# d = defaultdict(preprocessing.LabelEncoder)
le = preprocessing.LabelEncoder()
df_liar_train[categoricalVariables] = df_liar_train[categoricalVariables].applymap(str)

# encoded_data = df_liar_train[categoricalVariables].apply(lambda x: d[x.name].fit_transform(x))

In [ ]:
encoded_data = df_liar_train[categoricalVariables].apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')


In [ ]:
#encoding test dataset
df_liar_test[categoricalVariables] = df_liar_test[categoricalVariables].applymap(str)
encoded_test_data = df_liar_test[categoricalVariables].apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

In [ ]:
df_liar_train_enc = pd.concat([df_liar_train[numericalVariables], encoded_data], axis=1)
df_liar_test_enc = pd.concat([df_liar_test[numericalVariables], encoded_test_data], axis=1)

In [ ]:
df_liar_train_enc.shape

#  Data Visualization

Exploring the datasets to get a better understanding of the distribuitions

## Visualization

Word clouds help us visualize words with high frequencies in the dataset

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib inline

Training data word cloud

In [ ]:
def cloud(data,backgroundcolor = 'white', width = 800, height = 600):
    wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor,
                         width = width, height = height).generate(data)
    plt.figure(figsize = (15, 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()
    
cloud(' '.join(df_liar_train['clean']))

testing data word cloud

In [ ]:
cloud(' '.join(df_liar_test['clean']))

validation data word cloud

In [ ]:
cloud(' '.join(df_liar_val['clean']))

##  Visualizing the factors for feature: Verifiable Authenticity

In [ ]:
liar_col = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true',	'false',	'half_true',	'mostly_true',	'pants_on_fire',	'venue']

In [ ]:
df_liar_train = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/train_processed.csv")
df_liar_test = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/test_processed.csv")
df_liar_val = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/Rimzim/valid_processed.csv")

In [ ]:
df_liar_train.head()

In [ ]:
# df_liar_test = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/liar_dataset/test.tsv", names=liar_col, sep = "\t")
# df_liar_val = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/liar_dataset/valid.tsv", names=liar_col, sep = "\t")

In [ ]:
df_liar_train.info()

In [ ]:
#Liar Liar Dataset
label_distribution = df_liar_train.groupby("label").count()
label_distribution = label_distribution[["jsonid"]]
label_distribution.columns = ["Count"]
label_distribution = label_distribution.transpose()[['pants-fire', 'false', 'barely-true', 'half-true', 'mostly-true', 'true']].transpose()
ax = label_distribution.plot(kind='bar', figsize = (15, 5))
#label_distribution

Plotting various distribuitions

In [ ]:

def get_truth_distribution(data, col_name, count_threshold, normalize = True):
    df = data[[col_name, "jsonid", "label"]]
    df.columns = [col_name, "Percent", "label"]
    col_subject = df.groupby([col_name, "label"]).count()[["Percent"]]
    col_subject = col_subject.unstack('label')
    col_subject = col_subject.fillna(0)
    col_subject = col_subject[[('Percent', 'pants-fire'),
                               ('Percent', 'false'),
                               ('Percent', 'barely-true'),
                               ('Percent', 'half-true'),
                               ('Percent', 'mostly-true'),
                               ('Percent', 'true')]]
    col_subject = col_subject[col_subject.sum(axis = 1) >= count_threshold]
    if normalize == True:
        col_subject = col_subject.div(col_subject.sum(axis = 1),axis = 'index')
    col_subject = col_subject.sort_values(by=[('Percent', 'pants-fire')], ascending = False)
    return col_subject

Are statements in debates more likely to be lies, compared to other speeches?

In [ ]:
debate = df_liar_train[df_liar_train["context"] == "a debate."]
debate_distribution = get_truth_distribution(debate, "context", 10)

plt.style.use('ggplot')
groups = debate_distribution.columns.get_level_values(1).tolist()
ind = np.arange(len(groups))  # the x locations for the groups
ax = debate_distribution.plot(width=0.5,
                              kind='bar',
                              ec='black',
                              figsize=(8,5))

plt.title('Truth distribution for statements in debates')
plt.ylabel('Percentage')

ax.legend(groups)

 Compare distribution of fake news between traditional press, speeches and social media

In [ ]:
context_distribution = get_truth_distribution(df_liar_train, "context", 70, normalize = True) 
context_distribution.head()

**Social Media**

In [ ]:
social_media_regex = re.compile(r'\b(Facebook|blog|twitter|my space|newsletter|video|email|tweet|social media|YouTube)[s]*\b' ,re.I)
social_media_sets = context_distribution[context_distribution.index.str.contains(social_media_regex, regex=True) == True]
plt.style.use('ggplot')
groups = social_media_sets.columns.get_level_values(1).tolist()
ind = np.arange(len(groups))  # the x locations for the groups
ax = social_media_sets.plot(width=0.5,
                              kind='bar',
                              ec='black',
                              figsize=(8,5))

plt.title('Truth distribution in social media')
plt.ylabel('Percentage')

ax.legend(groups)

# Common Functions: for evaluation metrics

In [ ]:
# Function to get roc curve
def get_roc (y_test,y_pred):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    #Plot of a ROC curve
    plot.figure()
    lw = 2
    plot.plot(fpr, tpr, color='darkorange',
             label='ROC curve (area = %0.2f)' % roc_auc)
    plot.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plot.xlim([0.0, 1.0])
    plot.ylim([0.0, 1.0])
    plot.xlabel('False Positive Rate')
    plot.ylabel('True Positive Rate')
    plot.title('Receiver operating characteristic')
    plot.legend(loc="upper left")
    plot.show()
    return

# Common Classification and regression functions

We will create common functions for all algorithms, to be able to run them by just passing different training and testing data without code duplication -  - pass xtrain, ytrain, xtest, ytest as parameters to run the algorithms

Naive Bayes Classifier

In [ ]:
def NBClassifier(xtrain, ytrain, xtest, ytest):
  gnb = GaussianNB()
  gnb.fit(xtrain,ytrain)
  predicted = gnb.predict(xtest)
  print(classification_report(ytest, predicted))
  cnfmtx=confusion_matrix(ytest, predicted)
  print("Confusion matrix:\n",cnfmtx)

  print("Mean squared error: %.2f" %mean_squared_error(ytest, predicted))
  print("Mean absolute error: %.2f" %mean_absolute_error(ytest, predicted))
  print('Root Mean Squared Error: %.2f ' %np.sqrt(mean_squared_error(ytest, predicted)))
  print('Accuracy :', accuracy_score(ytest, predicted))
  get_roc(ytest, predicted)

**Decision Tree Classifier**

In [ ]:
def DTreesClassifier(xtrain, ytrain, xtest, ytest):
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(xtrain,ytrain)
  predicted=clf.predict(xtest)
  cnfmtx=confusion_matrix(ytest, predicted)
  print("Confusion matrix:\n",cnfmtx)

  print("Mean squared error: %.2f" %mean_squared_error(ytest, predicted))
  print("Mean absolute error: %.2f" %mean_absolute_error(ytest, predicted))
  print('Root Mean Squared Error: %.2f ' %np.sqrt(mean_squared_error(ytest, predicted)))
  print('Accuracy :', accuracy_score(ytest, predicted))
  get_roc(ytest, predicted)

**Support Vector Machines**




In [ ]:
def SVMClassifier(X_train,Y_train,X_test,Y_test):
  model = SVC()
  model.fit(X_train,Y_train)
  predicted = model.predict(X_test)
  print(classification_report(Y_test, predicted))
  cnfmtx=confusion_matrix(Y_test, predicted)
  print("Confusion matrix:\n",cnfmtx)

  print("Mean squared error: %.2f" %mean_squared_error(Y_test, predicted))
  print("Mean absolute error: %.2f" %mean_absolute_error(Y_test, predicted))
  print('Root Mean Squared Error: %.2f ' %np.sqrt(mean_squared_error(Y_test, predicted)))
  get_roc(Y_test, predicted)

**Linear Regression**

In [ ]:
def LinRegClassifier(X_train,Y_train,X_test,Y_test):
  regressor = LinearRegression()
  regressor.fit(X_train,Y_train)

  predicted = regressor.predict(X_test)
  print("Mean squared error: %.2f" %mean_squared_error(Y_test, predicted))
  print("Mean absolute error: %.2f" %mean_absolute_error(Y_test, predicted))
  print('Root Mean Squared Error: %.2f ' %np.sqrt(mean_squared_error(Y_test, predicted)))
  get_roc(Y_test, predicted)

Logistic Regression

In [ ]:
def LogRegression(X_train,Y_train,X_test,Y_test):
  logClassifier = linear_model.LogisticRegression(solver='liblinear', C=1, random_state=111)
  logClassifier.fit(X_train, Y_train)
  predicted = logClassifier.predict(X_test)

  from sklearn import metrics
  print("accuracy=", metrics.accuracy_score(Y_test, predicted))
  get_roc(Y_test, predicted)

# Converting characters to numerical vectors using Gensim Word2Vec 

In [ ]:
tokenstrain = [nl.word_tokenize(sentences) for sentences in df_liar_train['headline_text']]
tokenstest = [nl.word_tokenize(sentences) for sentences in df_liar_test['headline_text']]

headline = [nl.word_tokenize(sentences) for sentences in df_liar_train['headline_text']]
headlinetest =   [nl.word_tokenize(sentences) for sentences in df_liar_test['headline_text']]

In [ ]:
model = gensim.models.Word2Vec(tokenstrain, size=300, min_count=1, workers=4)
print("\n Training the word2vec model...\n")
model.train(tokens, total_examples=len(tokens), epochs=100)
model.wv.syn0


In [ ]:
model2 = gensim.models.Word2Vec(tokenstest, size=300, min_count=1, workers=4)
print("\n Training the word2vec model...\n")
model2.train(tokens, total_examples=len(tokens), epochs=100)
model2.wv.syn0


In [ ]:
df_liar_train_wv = pd.DataFrame(model.wv.syn0)
df_liar_test_wv = pd.DataFrame(model2.wv.syn0)

In [ ]:
df_liar_train_wv

In [ ]:
corpusLiar = []
for i in df_liar_train['headline_text']:
    corpusLiar.append(i)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer(max_features=1000)

tfidf_liar = tfidfVec.fit_transform(corpusLiar)

# 1. Feature Importance 



Random forests are among the most popular machine learning methods. They provide relatively good accuracy, robustness and ease of use. They also provide two methods for feature selection: **mean decrease impurity and mean decrease accuracy.**

---
Random forest consists of a number of decision trees. Every node in the decision trees is a condition on a single feature, designed to split the dataset into two so that similar response values end up in the same set.For classification, it is typically either Gini impurity or information gain/entropy and for regression trees it is variance.


In [ ]:
df_liar_train_enc.dropna(axis=0, inplace=True)

In [ ]:
df_liar_test_enc.dropna(axis=0, inplace=True)

In [ ]:
#reading our Liar Liar dataset into new variables to run Random Forests
xtrain_rf = df_liar_train_enc.iloc[:,3:-1]
ytrain_rf = df_liar_train_enc['label']
xtest_rf = df_liar_test_enc.iloc[:,3:-1]
ytest_rf = df_liar_test_enc['label']
colnames = ['headline_text', 'subject', 'speaker', 'speakerjobtitle', 'stateinfo','partyaffiliation', 'barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts','context', 'clean']


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# names=['barelytruecounts', 'falsecounts','halftruecounts','mostlytrueocunts','pantsonfirecounts']
rf = RandomForestRegressor()
rf.fit(xtrain_rf, ytrain_rf)
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), colnames), 
             reverse=True))

# MultiClass Classification

In [ ]:
# Convert text to word count vectors with CountVectorizer
# create the transform
cvec = CountVectorizer()

# tokenize, build vocab and encode training data
traindata_cvec = cvec.fit_transform(df_liar_train['headline_text'].values)

# summarize
print(cvec.vocabulary_)
# print(cvec.get_feature_names())

In [ ]:

# summarize encoded vector
print(traindata_cvec.shape)
print(type(traindata_cvec))
print(traindata_cvec.toarray())

In [ ]:

# import necessary packages
import pandas as pd
import numpy as np

import seaborn as sns

import nltk
from nltk.stem import PorterStemmer #for stemming
from nltk.tokenize import word_tokenize
import nltk.corpus

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import svm

from gensim.models.word2vec import Word2Vec

import matplotlib.pyplot as plt
import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:

# Calculate inverse document frequencies
# create the transform
tfidf_vec = TfidfTransformer()

# tokenize, build vocab and encode training data
traindata_tfidf_vec = tfidf_vec.fit_transform(traindata_cvec)

In [ ]:

# tfidf score
tfidf_vec.transform(traindata_cvec)

In [ ]:

# summarize encoded vector
print(traindata_tfidf_vec.shape)
print(traindata_tfidf_vec.toarray())

In [ ]:
# tfidf + ngrams
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), use_idf=True, smooth_idf=True)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_liar_train['headline_text'].values)

In [ ]:

# print(tfidf_ngram.toarray())
# tfidf.vocabulary_
first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0]


# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

In [ ]:

nltk.download('treebank')
nltk.download('universal_tagset')

In [ ]:
# POS tagging using CRF
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')
# print(len(tagged_sentence))

In [ ]:
training_sentences = df_liar_train['headline_text']

MultiClass Classification

In [ ]:
# using LogisticRegression
logReg_pipeline_cv = Pipeline([
    ('LogRCV', cvec),
    ('LogR_model', LogisticRegression())
])

logReg_pipeline_cv.fit(df_liar_train['headline_text'], df_liar_train['label'])
predictions_logReg = logReg_pipeline_cv.predict(df_liar_test['headline_text'])
logReg_cv = np.mean(predictions_logReg == df_liar_test['label'])

# using SVM
svm_pipeline_cv = Pipeline([
    ('svmCV', cvec),
    ('svm_model', svm.LinearSVC())
])

svm_pipeline_cv.fit(df_liar_train['headline_text'], df_liar_train['label'])
predictions_svm = svm_pipeline_cv.predict(df_liar_test['headline_text'])
svm_cv = np.mean(predictions_svm == df_liar_test['label'])

# using naive bayes
nb_pipeline_cv = Pipeline([
    ('nbCV', cvec),
    ('nb_model', MultinomialNB())
])

nb_pipeline_cv.fit(df_liar_train['headline_text'], df_liar_train['label'])
predictions_nb = nb_pipeline_cv.predict(df_liar_test['headline_text'])
nb_cv = np.mean(predictions_nb == df_liar_test['label'])

# using random forest
rf_pipeline_cv = Pipeline([
    ('rfCV', cvec),
    ('rf_model', RandomForestClassifier(n_estimators=400, n_jobs=4))
])

rf_pipeline_cv.fit(df_liar_train['headline_text'], df_liar_train['label'])
predictions_rf = rf_pipeline_cv.predict(df_liar_test['headline_text'])
rf_cv = np.mean(predictions_rf == df_liar_test['label'])

In [ ]:
print(logReg_cv)
print(svm_cv)
print(nb_cv)
print(rf_cv)

# Named Entity Recognition

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [ ]:
df_liar_train['TokenizedContext'] = df_liar_train['context'].apply(lambda x: preprocess(str(x))) 

In [ ]:
df_liar_train[['context', 'TokenizedContext']]
#part of speech tagging

In [ ]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

cp = nltk.RegexpParser(pattern)

def RegexpParser(sent):
  cs = cp.parse(sent)
  return cs

In [ ]:
df_liar_train['TokenizedContext'] = df_liar_train['TokenizedContext'].apply(lambda x: RegexpParser(x)) 

In [ ]:
from nltk.chunk import conlltags2tree, tree2conlltags

df_liar_train['iob_tagged_context'] = df_liar_train['TokenizedContext'].apply(lambda x: tree2conlltags(x)) 

In [ ]:
df_liar_train['new_context'] = df_liar_train['context'].apply(lambda x:  nltk.ne_chunk(pos_tag(word_tokenize(str(x))))) 

In [ ]:
df_liar_train[['iob_tagged_context', 'new_context', 'context']]

Use TF-Idf vectorizer to vectorize the data

# LDA Modelling and Topic Naming

In [ ]:
df_liar_train['index'] = df_liar_train.index
data = df_liar_train
train_lda = data[['clean', 'index']]
train_lda.head(4)

In [ ]:
df_liar_test['index'] = df_liar_test.index
data = df_liar_test
test_lda = data[['clean', 'index']]
test_lda.head(4)

In [ ]:
df_liar_val['index'] = df_liar_val.index
data = df_liar_val
valid_lda = data[['clean', 'index']]
valid_lda.head(4)

#### Split the clean news into list of words

In [ ]:
processed_docs = train_lda['clean'].map(lambda doc: doc.split(" "))
processed_docs[:4]

### Latent Dirichlet Allocation (LDA)

> It is an example of a probabilistic topic model. Topic models are a great way to automatically explore and structure a large set of documents: they group or cluster documents based on the words that occur in them. As documents on similar topics tend to use a similar sub-vocabulary, the resulting clusters of documents can be interpreted as discussing different 'topics'.

In [ ]:
def get_word_tokens(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if len(token) > 3:
            result.append(token)
    return result
tokenized_docs_local = df_liar_train['clean'].map(get_word_tokens)

### Create a function to build the dictionary and tokenized docs for given feature

Below function does the following
* #### Dictionary
Returns Dictionary given, dataframe and column name
* #### Tokenizeddocs
Returns Tokenizeddocs, of the all the words in a text in that column can be used for bow_corpus
* #### Dictionary is filtered using Gensim filter_extremes
    Filter out tokens that appear in less than 15 documents (absolute number) or more than 0.5 documents (fraction of total corpus size, not absolute number). after the above two steps, keep only the first 100000 most frequent tokens.


In [ ]:
def get_dictionary_print_words(dataframe,colname):
    dictionary_gensim = gensim.corpora.Dictionary(processed_docs)
    count = 0
    print('######## DICTIONARY Words and occurences ########')
    for k, v in dictionary_gensim.iteritems():
        print(k, v)
        count += 1
        if count > 10:
            break
    dictionary_gensim.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    return dictionary_gensim, tokenized_docs_local

#### Gensim filter_extremes

> Filter out tokens that appear less than 15 documents (absolute number) or more than 0.5 documents (fraction of total corpus size, not absolute number). after the above two steps, keep only the first 100000 most frequent tokens.

#### Create a function to build bow_corpus from dictionary and tokenized_docs

In [ ]:
def get_bow_corpus_print_sample(dataframe,colname):
    dictionary_gensim, tokenized_docs_local = get_dictionary_print_words(dataframe, colname)
    bow_corpus_local = [dictionary_gensim.doc2bow(doc) for doc in tokenized_docs_local]
    bow_doc_local_0 = bow_corpus_local[0]
    print('\n ######## BOW VECTOR FIRST ITEM ########')
    print(bow_doc_local_0)
    print('\n ######## PREVIEW BOW ########')
    for i in range(len(bow_doc_local_0)):
        print("Word {} (\"{}\") appears {} time.".format(bow_doc_local_0[i][0], 
                                               dictionary_gensim[bow_doc_local_0[i][0]], bow_doc_local_0[i][1]))
    return bow_corpus_local, dictionary_gensim

**Gensim doc2bow**

For each document we create a dictionary reporting how many words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier

#### Create a function to build tfidf_corpus from bow_corpus

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [ ]:
def get_tfidf_corpus_print_sample(bow_corpus_local):
    from gensim import corpora, models
    tfidf = models.TfidfModel(bow_corpus_local)
    tfidf_corpus_local = tfidf[bow_corpus_local]
    print('\n ######## TFIDF VECTOR FIRST ITEM ########')
    
    from pprint import pprint
    for doc in tfidf_corpus_local:
        pprint(doc)
        break
    return tfidf_corpus_local

#### Create a function to run ldamodel and print top 10 topics

In [ ]:
def get_lda_model_print_top_topics(bow_corpusforlda,numtopics,dictionaryforlda):
    lda_model = gensim.models.LdaMulticore(bow_corpusforlda, num_topics=numtopics, id2word=dictionaryforlda, passes=2, workers=2)
    lda_all_topics=lda_model.show_topics(num_topics=numtopics, num_words=10,formatted=False)
    lda_topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in lda_all_topics]

    #Below Code Prints Topics and Words
    for topic,words in lda_topics_words:
        print(str(topic)+ "::"+ str(words))
    return lda_model

#### Create a function to run ldamodel and print top 10 topics

In [ ]:
def get_lda_model_topics_topwords_print_top_topics(bow_corpusforlda,numtopics,dictionaryforlda):
    lda_model = gensim.models.LdaMulticore(bow_corpusforlda, num_topics=numtopics, id2word=dictionaryforlda, passes=2, workers=2, random_state=1)
    lda_all_topics=lda_model.show_topics(num_topics=numtopics, num_words=10,formatted=False)
    lda_topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in lda_all_topics]

    #Below Code Prints Topics and Words
    for topic,words in lda_topics_words:
        print(str(topic)+ "::"+ str(words))
    return lda_model,lda_topics_words

#### Create a function to enrich data with lda topics, lda topics score, top words

In [ ]:
def identify_topic_number_score_label_topwords(text,dictionary_local,lda_model_local,lda_topics_top_words_local):
    bow_vector_local = dictionary_local.doc2bow(get_word_tokens(text))
    topic_number_local, topic_score_local = sorted(
        lda_model_local[bow_vector_local], key=lambda tup: -1*tup[1])[0]
    #print (topic_number_local, topic_score_local)
    return pd.Series([topic_number_local, topic_score_local," ".join(lda_topics_top_words_local[int(topic_number_local)][1])])

#### Create a function that can enrich topic data to dataframe

In [ ]:
def update_lda_results_to_dataset(dataframe,topiccolnames,coltoapplylda,colnamedictionary,colnameldamodel, colnameldatopwords):
    dataframe[topiccolnames] = dataframe.apply(
    lambda row: identify_topic_number_score_label_topwords(
        row[coltoapplylda],colnamedictionary,colnameldamodel,
        colnameldatopwords), axis=1)
    return dataframe

### Bag of Words

#### Create a dictionary and tokens

> Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

#### Create a function to convert text to word tokens from cleaned dataset

In [ ]:
bow_corpus_headline, dictionary_headline = get_bow_corpus_print_sample(df_liar_train,
                                                                      'clean')

### Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [ ]:
lda_model_headline, lda_headline_topic_words = get_lda_model_topics_topwords_print_top_topics(
    bow_corpus_headline, 10 ,dictionary_headline)

#### Generate TF-IDF bow_corpus
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [ ]:
tfidf_corpus_headline = get_tfidf_corpus_print_sample(bow_corpus_headline)

### Running LDA model using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

**GOAL**: To get top ten topics with top words. 

In [ ]:
lda_tfidf_model_headline  = get_lda_model_print_top_topics(tfidf_corpus_headline,10,dictionary_headline)

#### Explanation for LDA 
![[Explanation of LDA](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1508239587/n4ZpIXl_egq7mq.png)](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1508239587/n4ZpIXl_egq7mq.png)

### Semisupervised Labeling
Based on train,test and valid data explored the topic scores for sample data and identified below topics

In [ ]:
semisupervised_topic_labels = ['topic0','topic1','topic2','topic3','topic4','topic5','topic6','topic7','topic8','topic9']

####  Function to add topicnumber, topicscore, topiclabel, topwords

In [ ]:
headlinetopiccolnames = ['topic_number','lda_score','topic_top_words']
df_liar_train = update_lda_results_to_dataset(
    df_liar_train, headlinetopiccolnames,'clean', dictionary_headline, lda_model_headline, lda_headline_topic_words)
df_liar_train.head()

In [ ]:
df_liar_test = update_lda_results_to_dataset(
    df_liar_test,headlinetopiccolnames,'clean',
  dictionary_headline,lda_model_headline,lda_headline_topic_words)
df_liar_test.head()

In [ ]:
df_liar_val = update_lda_results_to_dataset(
    df_liar_val,headlinetopiccolnames,'clean',
  dictionary_headline,lda_model_headline,lda_headline_topic_words)
df_liar_val.head()

### Visualize the LDA Distribution of news against Top 10 Topics

**GOAL 1:** *Each of the N documents will be represented in the LDA model by a vector of length M*
**GOAL 2:** *Each of the M topics is represented by a vector of length V*

In [ ]:
import seaborn as sb
def create_distribution(dataFile):
    g = sb.countplot(x='topic_number', data=dataFile, palette='hls')
    g.set_xticklabels(g.get_xticklabels(),rotation=90)

    return g

create_distribution(df_liar_train) # TRAIN Document Vector

In [ ]:
create_distribution(df_liar_test)# TEST Document Vector

In [ ]:
create_distribution(df_liar_val)# VALID Document Vector

##### Saved the latest dataset into a seperate CSV file 

In [ ]:
train_news.to_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/train_lda.csv", sep=',')
test_news.to_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/test_lda.csv", sep=',')
valid_news.to_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/valid_lda.csv", sep=',')

## Let us check the accuracies using the LDA score as X and label as Y

In [ ]:
true_labels = ['original','true','mostly-true','half-true']
false_labels = ['barely-true','false','pants-fire']
def simplify_label(input_label):
    if input_label == 'original':
        return 1
    elif input_label == 'true':
        return 1
    elif input_label == 'mostly-true':
        return 1
    elif input_label == 'half-true':
        return 0
    elif input_label == 'barely-true':
        return 0
    elif input_label == 'false':
        return 0
    elif input_label == 'pants-fire':
        return 0

labelcolname = 'encoded_label'

df_liar_train[labelcolname] = df_liar_train.apply(lambda row: simplify_label(row['label']), axis=1)
df_liar_test[labelcolname] = df_liar_test.apply(lambda row: simplify_label(row['label']), axis=1)

In [ ]:
X_train = df_liar_train['lda_score'].values.reshape(-1, 1)
Y_train = df_liar_train['encoded_label'].values.reshape(-1, 1)
X_test = df_liar_test['lda_score'].values.reshape(-1, 1)
Y_test = df_liar_test['encoded_label'].values.reshape(-1, 1)

 **Naive Bayes Classifier**

In [ ]:
import matplotlib.pyplot as plot

NBClassifier(X_train, Y_train, X_test, Y_test)

**Decision Tree Classifier**

In [ ]:
DTreesClassifier(X_train, Y_train, X_test, Y_test)

**Support Vector Machines**




In [ ]:
SVMClassifier(X_train, Y_train, X_test, Y_test)

**Linear Regression**

In [ ]:
LinRegClassifier(X_train, Y_train, X_test, Y_test)

**Logistic Regression**

In [ ]:
LogRegression(X_train, Y_train, X_test, Y_test)

# Amalgamation of Liar Liar and Kaggle Fake News dataset - using Cosine Similarity

In [ ]:
#Kaggle fake news dataset
df_fake_train = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/fake_news/train.csv")
df_fake_test = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/fake_news/test.csv")
df_fake_sub = pd.read_csv("/content/gdrive/My Drive/MLSpring2020/datatribe-nutritionbasediseaseprediction/AlternusVera/Datasets/fake_news/submit.csv")

In [ ]:
df_fake_train.text[0]

In [ ]:
df_liar_train.headline_text[0]

In [ ]:

corpusLiar = []
for i in df_liar_train['headline_text']:
    corpusLiar.append(i)

corpusFake = []
for i in df_fake_train['text'].astype('U'):
    corpusFake.append(i)  

Train a classifier that predicts the fakeness in the range of classes given by liar liar dataset

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer(max_features=1000)

tfidf_liar = tfidfVec.fit_transform(corpusLiar)

In [ ]:
tf_idf_fakenews = tfidfVec.fit_transform(corpusFake)
words = tfidfVec.get_feature_names()

In [ ]:
tfidf_liar.toarray()

In [ ]:
tf_idf_fakenews.toarray()

In [ ]:
tfidfVec.vocabulary_

**Cosine Similarity**

Liar Liar dataset

In [ ]:
train_tfidf = tfidfVec.fit_transform(df_liar_train['headline_text'])
print('Training dim:', train_tfidf.shape)
print(train_tfidf.A[:10])


# test_tfidf = tfidfVec.fit_transform(df_liar_test['statement'])
# print('Test dim:', test_tfidf.shape)
# print(test_tfidf.A[:10])


cosine similarity of Training data for liar liar dataset

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
similarity_score = []
for i in range(len(train_tfidf.toarray())):
    similarity_score.append(1 - spatial.distance.cosine(tf_idf_fakenews[0].toarray(), tfidf_liar[i].toarray()))

In [ ]:
#append the similarity_score to both liar liar dataset and kaggle fake news dataset for amalgamation
df_liar_train['cosine_score'] = similarity_score


Kaggle fake news dataset

In [ ]:
similarity_score_fake = []
for i in range(len(tf_idf_fakenews.toarray())):
    similarity_score_fake.append(1 - spatial.distance.cosine(tfidf_liar[0].toarray(), tf_idf_fakenews[i].toarray()))

In [ ]:
df_fake_train['cosine_score'] = similarity_score_fake

In [ ]:
df_liar_train = df_liar_train.dropna(0)

In [ ]:
df_liar_train.isnull().sum()

In [ ]:
df_fake_train = df_fake_train.dropna(0)

In [ ]:
df_liar_train.shape

In [ ]:
df_fake_train.shape

In [ ]:
df_liar_train.head()

Amalgamtion of the two datasets - liar liar and kaggle fake news

In [ ]:
def simplify_label(input_label):
    if input_label in true_labels:
        return 1
    else:
        return 0

In [ ]:
df_amal = df_liar_train.merge(df_fake_train, on="cosine_score", how="inner")

In [ ]:
df_amal.dropna(axis=0, inplace=True)

# Data Enrichment - Adding a new column called Authenticity

This distillation step involves creating a new column of authenticity based on the source of the statement.

In [ ]:
true_labels = ['original','true','mostly-true','half-true']
false_labels = ['barely-true','false','pants-fire']
def simplify_label(input_label):
    if input_label in true_labels:
        return 1
    else:
        return 0

In [ ]:
labelcolname = 'encoded_label'

df_liar_train[labelcolname] = df_liar_train.apply(lambda row: simplify_label(row['label']), axis=1)
df_liar_test[labelcolname] = df_liar_test.apply(lambda row: simplify_label(row['label']), axis=1)
df_liar_val[labelcolname] = df_liar_val.apply(lambda row: simplify_label(row['label']), axis=1)

In [ ]:
true_labels = ['news','interview','television','show', 'speech', 'reporters', 'debate', 'newsletter', 'press', 'CNN', 'ABC', 'CBS', 'video', 'conference', 'official', 'book']
false_labels = ['website', 'tweet', 'mail', 'e-mail', 'mailer', 'web', 'site', 'meme', 'comic', 'advertisement', 'ad', 'blog', 'flier', 
                'letter', 'social', 'tweets', 'internet', 'message', 'campaign', 'post', 'facebook', 'handout', 'leaflet', 'letter' ]
def simplify_venue_label(input_label):
    words = input_label.split(" ")
    for s in words:
      if s in true_labels:
        return 1
      elif s in false_labels:
        return 0
    else:
        return 1

In [ ]:
labelcolname = 'authenticity'

df_liar_train[labelcolname] = df_liar_train.apply(lambda row: simplify_venue_label(str(row['context'])), axis=1)
df_liar_test[labelcolname] = df_liar_test.apply(lambda row: simplify_venue_label(str(row['context'])), axis=1)
df_liar_val[labelcolname] = df_liar_val.apply(lambda row: simplify_venue_label(str(row['context'])), axis=1)


In [ ]:
obj = [df_liar_train, df_liar_val]
df_train = pd.concat(obj)

In [ ]:
X_train = df_train['authenticity'].values.reshape(-1, 1)
Y_train = df_train['encoded_label'].values
X_test = df_liar_test['authenticity'].values.reshape(-1, 1)
Y_test = df_liar_test['encoded_label'].values.reshape(-1, 1)

## running the classification algorithms with this newly added column as X and label as Y

 **Naive Bayes Classifier**

In [ ]:
NBClassifier(X_train, Y_train, X_test, Y_test)

**Decision Tree Classifier**

In [ ]:
DTreesClassifier(X_train, Y_train, X_test, Y_test)

**Support Vector Machines**




In [ ]:
SVMClassifier(X_train, Y_train, X_test, Y_test)

**Linear Regression**

In [ ]:
LinRegClassifier(X_train, Y_train, X_test, Y_test)

**Logistic Regression**

In [ ]:
LogRegression(X_train, Y_train, X_test, Y_test)

# Polynomial Equation

In [ ]:
  X_train = df_train['authenticity'].values.reshape(-1, 1)
  Y_train = df_train['encoded_label'].values
  X_test = df_liar_test['authenticity'].values.reshape(-1, 1)
  Y_test = df_liar_test['encoded_label'].values.reshape(-1, 1)
  
  model = SVC()
  model.fit(X_train,Y_train)
  predicted = model.predict(X_test)
  print(classification_report(Y_test, predicted))

In [ ]:
def DATATRIBE_getAuthenticityScoreBySource(src):
  x = simplify_venue_label(src)
  xTrain = np.array(x).reshape(-1, 1)

  xPpredicted = model.predict(xTrain)
  xPredicedProb = logClassifier.predict_proba(xTrain)[:,1]
  return 1 - float(xPredicedProb)

print(DATATRIBE_getAuthenticityScoreBySource('news'))

In [ ]:

def isFakeNews(headline="", source = "", speaker ="",context =""):
    accur = [0.58, 0.52, 0.56] # using the (normalized) accuracy as weigths
    w = [float(i)/sum(accur) for i in accur]
    sumW = 0
    prob = []
    # if ( (headline != "") & (speaker != "")):
    #     prob.append(w[0] * DATATRIBE_getEventCoverageScore(headline,speaker))
    #     sumW += w[0]
    # if ((context != "") & (speaker != "")):
    #     prob.append(w[1] * DATATRIBE_getReliableSource(context,speaker))
    #     sumW += w[1]
    if (source != ""):
        prob.append(w[2] * DATATRIBE_getAuthenticityScoreBySource(source))
        sumW += w[2]
    probTotal = sum(prob[0:len(prob)]) / sumW
    return probTotal
    
result = isFakeNews("Says the Annies List political group supports third-trimester abortions on demand.", "news", "barack-obama","a news release")
print(result)
if result > 0.5:
    print("is FAKE NEWS!!!")
else:
    print("it is NOT fake news!!!")

# Conclusion

**Multiclass classification results**
We first modeled the data with a baseline model and the accuracies obtained are as below:
- Logistic Regression: 0.24230465666929754
- SVM: 0.23914759273875297
- Naive Bayes: 0.24230465666929754
- Random Forests0.27466456195737965


 we added a new feature called authenticity to our dataset which we enriched manually. This gave the below results:
- Naive Bayes= 0.58
- Decision Trees = 0.56
- SVM = 0.55
- Regression = 0.56

Next, we tried the process of distillation like LDA, NER and amalgamation with Kaggle fake news dataset to enrich our dataset. The accuracies improved to these scores:
- Naive Bayes= 0.65
- Decision Trees = 0.55
- SVM = 0.55
- Regression = 0.66

Thus, we can see that multiclass classification benefits from enrichments made to the dataset. Processes like distillation, NER, LDA, word embeddings help to bring out the important features and model better and improve accuracy

